In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import datetime
import kerastuner as kt

In [ ]:
# Problem definition

def f(x):
    return np.log(x)


Domain = (0.5, 10)
# Generate training data
x_train = np.linspace(*Domain, 10000)
y_train = f(x_train)


# Generate test data
x_test = np.linspace(*Domain, 1000)
y_test = f(x_test)

In [ ]:
# Maually define the model

def create_model(layers, activation = 'relu', optimizer = 'adam', loss = 'mse'):
    model = keras.Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(keras.layers.Dense(nodes, activation = activation, input_shape = (1,)))
        else:
            model.add(keras.layers.Dense(nodes, activation = activation))
    model.add(keras.layers.Dense(1, activation = 'linear'))
    model.compile(optimizer = optimizer, loss = loss, metrics = ['mse'])
    return model

model = create_model([10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [ ]:
def create_hypermodel(hp):
    LR = hp.Choice('learning_rate', [0.005, 0.001, 0.0005, 0.0001])
    # DROPOUT_RATE = hp.Float('dropout_rate', 0.0, 0.5, step=0.1, default=0.0)
    NUM_DIMS = hp.Int('num_dims', 16, 64, step=8)
    NUM_LAYERS = hp.Int('num_layers', 2, 16, step=2)

    model = keras.Sequential()
    model.add(keras.layers.Dense(NUM_DIMS, activation='relu', input_shape=(1,)))
    for _ in range(NUM_LAYERS):
        model.add(keras.layers.Dense(NUM_DIMS, activation='relu'))
        # model.add(keras.layers.Dropout(DROPOUT_RATE))
    model.add(keras.layers.Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(LR), metrics=['mse'])

    return model


def train_model(model, x_train, y_train, x_test, y_test, epochs = 10, batch_size = 32):
    history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = epochs, batch_size = batch_size
                        , callbacks = [
        # keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3), 
        keras.callbacks.TensorBoard(log_dir = 'logs/{}'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))])
    return history


use_hypermodel = True


# Create the model
tuner = kt.Hyperband(create_hypermodel,
                        objective='mse',
                        max_epochs=10,
                        factor=3,
                        directory='my_dir',
                        project_name='intro_to_kt-log')


In [ ]:
# Train the model
if use_hypermodel:
    tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
    best_hps = tuner.get_best_hyperparameters()[0]
    print('Best hyperparameters:', best_hps.values)

    model = tuner.hypermodel.build(best_hps)
    history = train_model(model, x_train, y_train, x_test, y_test, epochs = best_hps.values['tuner/epochs'])

else:
    history = train_model(model, x_train, y_train, x_test, y_test)


# Evaluate the model
# test_acc = model.evaluate(x_test,  y_test, verbose=2)

# print('\nTest accuracy:', test_acc)


In [ ]:
# Show the evaluation results
# with two subplots: one for the direct comparison of the predicted and the actual values
# and one for the error distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# Plot the predicted and the actual values
ax1.plot(x_test, y_test, label='Actual')
ax1.plot(x_test, model.predict(x_test), label='Predicted')
ax1.set_title('Actual vs. predicted values')
ax1.legend()

# Plot the error distribution
error = np.abs(model.predict(x_test).T - y_test)
ax2.plot(x_test, error.T, label='Error')
ax2.set_title('Error distribution')
ax2.set_xlabel('Error')
ax2.set_ylabel('Count')

plt.show()

In [ ]:
model.summary()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
!kill 30058